In [227]:
import pandas as pd
from openai import OpenAI
import openai
import os

In [228]:
dataset_path = 'Housing.csv'
client = OpenAI()

In [229]:
# Initialize the dataframe through the dataset.

df = pd.read_csv(dataset_path)

houses = []

# Convert the whole dataframe into array of dictionaries. Where each record is the dictionary. 

houses = df.to_dict(orient='records')

In [230]:
# Dividing records into categories that will be used further for prompt construction.

affordable_houses = [record for record in houses if 0 <= record.get("price", 0) <= 2000000]
moderate_houses = [record for record in houses if 2000000 <= record.get("price", 0) <= 4000000]
aboveAverage_houses = [record for record in houses if 4000000 <= record.get("price", 0) <= 6000000]
highEnd_houses = [record for record in houses if 6000000 <= record.get("price", 0) <= 8000000]
luxury_houses = [record for record in houses if 8000000 <= record.get("price", 0) <= 10000000]
premiumLuxury_houses = [record for record in houses if 10000000 <= record.get("price", 0) <= 12000000]

In [231]:
houses_for_post = [] # Array of houses that will be used to generate posts.

# Applying filtering with specific feature.
for record in affordable_houses:
    if record['mainroad'] == 'yes' and record['area'] >= 3000 and record['bedrooms'] >= 3:
        if (record['bedrooms'] >= 3):
            filtered_record = {'affordable_houses': record}
            houses_for_post.append(filtered_record)

# Applying filtering with specific feature.
for record in moderate_houses:
    if record['mainroad'] == 'yes' and record['area'] >= 3000 and record['bedrooms'] >= 3:
        if record['basement'] == 'yes' and record['airconditioning'] == 'yes' and record['guestroom'] == 'yes':
            filtered_record = {'moderate_houses': record}
            houses_for_post.append(filtered_record)

# Applying filtering with specific feature.
for record in aboveAverage_houses:
    if record['mainroad'] == 'yes' and record['area'] >= 3000 and record['bedrooms'] >= 3:
        if record['basement'] == 'yes' and record['airconditioning'] == 'yes' and record['guestroom'] == 'yes':
            if record['furnishingstatus'] == 'furnished' and record['parking'] >= 1 and record['prefarea'] == 'yes':
                    filtered_record = {'aboveAverage_houses': record}
                    houses_for_post.append(filtered_record)

# Applying filtering with specific feature.
for record in highEnd_houses:
    if record['mainroad'] == 'yes' and record['area'] >= 3000 and record['bedrooms'] >= 3:
        if record['basement'] == 'yes' and record['airconditioning'] == 'yes' and record['guestroom'] == 'yes':
            if record['furnishingstatus'] == 'furnished' and record['parking'] >= 1 and record['prefarea'] == 'yes':
                    filtered_record = {'highEnd_houses': record}
                    houses_for_post.append(filtered_record)

# Applying filtering with specific feature.
for record in luxury_houses:
    if record['mainroad'] == 'yes' and record['area'] >= 3000 and record['bedrooms'] >= 3:
        if record['basement'] == 'yes' and record['airconditioning'] == 'yes' and record['guestroom'] == 'yes':
            if record['furnishingstatus'] == 'furnished' and record['parking'] >= 1 and record['prefarea'] == 'yes':
                    filtered_record = {'luxury_houses': record}
                    houses_for_post.append(filtered_record)

# Applying filtering with specific feature.
for record in premiumLuxury_houses:
    if record['mainroad'] == 'yes' and record['area'] >= 3000 and record['bedrooms'] >= 3:
        if record['basement'] == 'yes' and record['airconditioning'] == 'yes' and record['guestroom'] == 'yes':
            if record['furnishingstatus'] == 'furnished' and record['parking'] >= 1 and record['prefarea'] == 'yes':
                    filtered_record = {'premiumLuxury_houses': record}
                    houses_for_post.append(filtered_record)

In [232]:
# Again dividing the data about filtered houses by specific features to the variable data context.
affordable_houses_data = houses_for_post[0].get('affordable_houses')
moderate_houses_data = houses_for_post[1].get('moderate_houses')
aboveAverage_houses_data = houses_for_post[2].get('aboveAverage_houses')
highEnd_houses_data = houses_for_post[3].get('highEnd_houses')
luxury_houses_data1 = houses_for_post[4].get('luxury_houses')
luxury_houses_data2 = houses_for_post[5].get('luxury_houses')

#Prompts that used in generating posts.

prompt_affordable_houses = f"Your mission is to make a positive and quote-provoking post, describing each data point about house. Moreover, you need enhance your post with unique approach in marketing or selling. Here are the data:\n{affordable_houses_data}\nGenerate a response:"
prompt_moderate_houses = f"Your mission is to make a positive and quote-provoking post, describing each data point about house. Moreover, you need enhance your post with unique approach in marketing or selling. Here are the data:\n{moderate_houses_data}\nGenerate a response:"
prompt_aboveAverage_houses = f"Your mission is to make a positive and quote-provoking post, describing each data point about house. Moreover, you need enhance your post with unique approach in marketing or selling. Here are the data:\n{aboveAverage_houses_data}\nGenerate a response:"
prompt_highEnd_houses = f"Your mission is to make a positive and quote-provoking post, describing each data point about house. Moreover, you need enhance your post with unique approach in marketing or selling. Here are the data:\n{highEnd_houses_data}\nGenerate a response:"
prompt_luxury_houses1 = f"Your mission is to make a positive and quote-provoking post, describing each data point about house. Moreover, you need enhance your post with unique approach in marketing or selling. Here are the data:\n{luxury_houses_data1}\nGenerate a response:"
prompt_luxury_houses2 = f"Your mission is to make a positive and quote-provoking post, describing each data point about house. Moreover, you need enhance your post with unique approach in marketing or selling. Here are the data:\n{luxury_houses_data2}\nGenerate a response:"

In [257]:
# Generation of the post according to the role of context, and the main prompt from user.

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
   messages=[
    {"role": "system", "content": "You are the real estate agent who manage own instagram account for promoting houses to acquiring."},
    {"role": "user", "content": prompt_affordable_houses}
  ]
)

generated_post_affordable_houses = completion.choices[0].message.content

In [258]:
# Generation of the post according to the role of context, and the main prompt from user.


completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
   messages=[
    {"role": "system", "content": "You are the real estate agent who manage own instagram account for promoting houses to acquiring."},
    {"role": "user", "content": prompt_moderate_houses}
  ]
)

generated_post_moderate_houses = completion.choices[0].message.content

In [259]:
# Generation of the post according to the role of context, and the main prompt from user.

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
   messages=[
    {"role": "system", "content": "You are the real estate agent who manage own instagram account for promoting houses to acquiring."},
    {"role": "user", "content": prompt_aboveAverage_houses}
  ]
)

generated_post_aboveAverage_houses = completion.choices[0].message.content

In [260]:
# Generation of the post according to the role of context, and the main prompt from user.

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
   messages=[
    {"role": "system", "content": "You are the real estate agent who manage own instagram account for promoting houses to acquiring."},
    {"role": "user", "content": prompt_highEnd_houses}
  ]
)

generated_post_highEnd_houses = completion.choices[0].message.content

In [261]:
# Generation of the post according to the role of context, and the main prompt from user.

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
   messages=[
    {"role": "system", "content": "You are the real estate agent who manage own instagram account for promoting houses to acquiring."},
    {"role": "user", "content": prompt_luxury_houses1}
  ]
)

generated_post_luxury_houses1 = completion.choices[0].message.content

In [254]:
# Generation of the post according to the role of context, and the main prompt from user.

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
   messages=[
    {"role": "system", "content": "You are the real estate agent who manage own instagram account for promoting houses to acquiring."},
    {"role": "user", "content": prompt_luxury_houses2}
  ]
)

generated_post_luxury_houses2 = completion.choices[0].message.content

In [264]:
# This part of code, exists for generating md file with results

markdown_content = """# My Markdown File

This is some text in my Markdown file. You can include various formatting options such as:

- Lists
- *Italic text*
- **Bold text**
- [Links](https://www.example.com)
"""

file_path = "example.md"

# There is generation with divider - '-----'.

with open(file_path, "w", encoding="utf-8") as file:
    file.write(str(generated_post_affordable_houses) + "\n---\n")
    file.write(str(generated_post_moderate_houses) + "\n---\n")
    file.write(str(generated_post_aboveAverage_houses) + "\n---\n")
    file.write(str(generated_post_highEnd_houses) + "\n---\n")
    file.write(str(generated_post_luxury_houses1) + "\n---\n")
    file.write(str(generated_post_luxury_houses2) + "\n---\n")

print(f"Markdown file '{file_path}' has been generated.")

Markdown file 'example.md' has been generated.


In [265]:
# Converting md to txt. 

md_file_path = "example.md"
txt_file_path = "example.txt"

with open(md_file_path, "r", encoding="utf-8") as md_file:
    markdown_content = md_file.read()

with open(txt_file_path, "w", encoding="utf-8") as txt_file:
    txt_file.write(markdown_content)

print(f"Markdown file '{md_file_path}' has been converted to text file '{txt_file_path}'.")

Markdown file 'example.md' has been converted to text file 'example.txt'.
